
# Business Cases with Data Science 

## Case 2: Market Basket Analysis

#### Semester: Spring 2020
#### Group (Palm & Company) Member:
1. Pedro Santos (M20190420)
2. Ana Claudia Alferes (M20190932)
3. Lennart Dangers (M20190251)
4. Michael Machatschek (M20190054)

#### This project follows the CRISP-DM Process:

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modelling
5. Evaluation
6. Deployment

This jupypter-notebook provides the code in python to do a market basket analysis of a given dataset. A more detailed description of this project and the solutions can be found in the report. 

# Step 1: Business Understanding

Please consult our report for further details. 

# Step 2: Data Understanding

## Dataset Description

A data dictionary is available in the appendix of our report.
 
 
 
 #### The following code will help to explore and understand the entire dataset. 

In [8]:
# Import packages
import csv
import pandas as pd
import numpy as np
import datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as go

In [3]:
# Load and show dataset sample (Chain of Asian Restaurant sales January 2018)
dtypes = {'DocNumber':'category','ProductDesignation':'category'}
df_par = pd.DataFrame(pd.read_csv("AsianRestaurant_Cyprus_2018_partial.txt",sep=";", dtype=dtypes))
df = pd.DataFrame(pd.read_csv("AsianRestaurant_Cyprus_2018.txt", sep=";"))

In [7]:
df.head()

,DocNumber,ProductDesignation,ProductFamily,Qty,TotalAmount,InvoiceDateHour,EmployeeID,IsDelivery,Pax,CustomerID,CustomerCity,CustomerSince
0,110000012018,SWEETCORN SOUP,SOUPS,2.0,"12,00",2018-01-01 21:47:27.000,2,0,2,0,NaN,NaN
1,110000012018,SPRING ROLL,STARTERS,2.0,"5,80",2018-01-01 21:47:27.000,2,0,2,0,NaN,NaN
2,110000012018,EGG FRIED RICE,RICE,1.0,"6,90",2018-01-01 21:47:27.000,2,0,2,0,NaN,NaN
3,110000012018,SWEET SOUR CHICKEN,MEAT,1.0,"12,60",2018-01-01 21:47:27.000,2,0,2,0,NaN,NaN
4,110000012018,LAMB ROGAN JOSH,MAIN IND,1.0,"14,60",2018-01-01 21:47:27.000,2,0,2,0,NaN,NaN


In [6]:
df.isnull().sum()

DocNumber                 0
ProductDesignation        0
ProductFamily             0
Qty                       0
TotalAmount               0
InvoiceDateHour           0
EmployeeID                0
IsDelivery                0
Pax                       0
CustomerID                0
CustomerCity          52861
CustomerSince         54967
dtype: int64